<div style="position: relative; text-align: center; padding: 30px;">
  <h1><strong>Comparasión de Clasificadores</strong></h1>
  <h3><strong>Walter y Heri</strong></h3>
</div>

**Objetivo:** El objetivo de esta actividad es que cada equipo cree una función que permita manejar diferentes formas de representación de clasificaciones.

**Instrucciones:**
- Construye una clase en Python que reciba una clasificación en representación vectorial, y construya la representación matricial y de conjunto de conjuntos de la clasificación. Haz un método para cada conversión:  
    - vectorial -> matricial.  
    - vectorial -> conjunto de conjuntos.  
- Guarda las tres representaciones como variables ocultas usando la nomenclatura "__parameter".  
- Crea tres métodos getter para devolver cada representación.  
- Construye un método para calcular los siguientes valores:  
    - Entropía de una clasificación.  
    - La entropía condicional de una clasificación dada otra clasificación.  
    - La información mutua de una clasificación dada otra clasificación.  
    - La variación de información de una clasificación dada otra clasificación.  